In [ ]:
ls

drive/  sample_data/


In [ ]:
cd drive/MyDrive/dacon/천체 유형 분류 대회

/content/drive/MyDrive/dacon/천체 유형 분류 대회


[Keras](https://wikidocs.net/32105)  
[Keras로 MLP하기](https://iostream.tistory.com/111)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

import sklearn
from sklearn.preprocessing import StandardScaler # 스케일링 함수

In [ ]:
train = pd.read_csv('train.csv', index_col='id')

In [ ]:
train.head()

,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
id,,,,,,,,,,,,,,,,,,,,,,
0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,21.387653,21.572827,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340
1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,20.937731,21.063646,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314
2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,17.095999,17.076449,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928
3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,20.073852,19.794505,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855
4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,19.861718,18.810375,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655


In [ ]:
unique_labels = train['type'].unique()  # type 종류 파악
label_dict = {val: i for i, val in enumerate(unique_labels)} # 번호를 주고 dict으로
i2lb = {v:k for k, v in label_dict.items()}

In [ ]:
scaler = StandardScaler()
labels = train['type']
train = train.drop(columns=['fiberID', 'type']) # fiberID는 1000개의 categorical feature이며, 이 커널에서는 무시합니다.

In [ ]:
_mat = scaler.fit_transform(train) # 학습용 데이터
train = pd.DataFrame(_mat, columns=train.columns, index=train.index)

scaler는 fit과 transform 메서드를 지니고 있습니다. fit 메서드로 데이터 변환을 학습하고, transform 메서드로 실제 데이터의 스케일을 조정합니다. 

이때, fit 메서드는 학습용 데이터에만 적용해야 합니다. 그 후, transform 메서드를 학습용 데이터와 테스트 데이터에 적용합니다. scaler는 fit_transform()이란 단축 메서드를 제공합니다. 학습용 데이터에는 fit_transform()메서드를 적용하고, 테스트 데이터에는 transform()메서드를 적용합니다.

In [ ]:
train_x = train
train_y = labels.replace(label_dict)

In [ ]:
train_x

,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
id,,,,,,,,,,,,,,,,,,,,
0,0.002522,0.017736,0.022913,0.026865,0.028350,0.002812,0.003432,0.028019,0.026184,0.024489,0.000844,0.019287,0.030818,0.023734,0.019085,0.021572,0.018062,0.023754,0.025371,0.029607
1,0.002373,0.013079,0.017916,0.022808,0.022707,0.002641,0.002612,0.020351,0.022507,0.020998,-0.000604,0.015060,0.024765,0.020265,0.020656,0.011300,0.013691,0.019199,0.022876,0.024857
2,0.002071,-0.012530,-0.013542,-0.011566,-0.008257,0.001755,-0.002722,-0.023707,-0.008885,-0.006338,-0.005030,-0.011112,-0.018435,-0.006467,-0.006246,-0.018749,-0.011420,-0.011496,-0.007869,-0.003743
3,0.002319,0.008782,0.012001,0.015437,0.014545,0.002442,0.001661,0.012031,0.015448,0.012297,-0.001778,0.009973,0.015467,0.014165,0.012306,0.005394,0.009011,0.012750,0.015747,0.017680
4,0.002796,0.028203,0.024150,0.012532,0.005864,0.003560,0.006081,0.028965,0.013715,0.005550,0.005094,0.025735,0.032826,0.013198,0.004744,0.038971,0.028465,0.024583,0.013683,0.008874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,0.002305,0.011340,0.012975,0.016956,0.018557,0.002421,0.002208,0.013072,0.017487,0.015154,-0.001269,0.013488,0.016917,0.016580,0.014700,0.005497,0.011022,0.013677,0.018063,0.019997
199987,0.002697,0.026585,0.020991,0.018385,0.015615,0.003794,0.005000,0.018342,0.014533,0.009983,0.008940,0.024592,0.016559,0.008915,0.007738,0.055691,0.021073,0.011903,0.009892,0.010016
199988,0.002556,0.020765,0.015178,0.003550,-0.002722,0.003229,0.004291,0.017516,0.005125,-0.001912,0.005250,0.023831,0.019006,0.005632,-0.001967,0.035916,0.020734,0.016711,0.005723,0.001066


In [ ]:
train_y

id
0         0
1         0
2         0
3         0
4         1
         ..
199986    0
199987    6
199988    1
199989    6
199990    0
Name: type, Length: 199991, dtype: int64

In [ ]:
# 학습률 설정? 0.0003으로 지정
# Adam 알고리즘을 사용 / 왜 Adam을 썼나?
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0003,
)

[학습률 설정](https://forensics.tistory.com/28)  
[옵티마이저](https://hiddenbeginner.github.io/deeplearning/2019/09/22/optimization_algorithms_in_deep_learning.html)  
[경사하강법](https://seamless.tistory.com/38)  
  
옵티마이저 (Optimizer)는 손실 함수을 통해 얻은 손실값으로부터 모델을 업데이트하는 방식

In [ ]:
model = tf.keras.models.Sequential([   # 가장 간단한 신경망 모델
  tf.keras.layers.Input(len(train_x.columns)),
  tf.keras.layers.Dense(256*4, activation='elu'), # 계층 / 신경망을 만드는 것? / units 갯수
  tf.keras.layers.Dropout(0.1), # 정규화 dropout / 인풋의 10%를 drop
  tf.keras.layers.Dense(256*4, activation='elu'),
  tf.keras.layers.Dense(256*3, activation='elu'),
  tf.keras.layers.Dropout(0.15),
  tf.keras.layers.Dense(256*3, activation='elu'),
  tf.keras.layers.Dense(256*2, activation='elu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(256*2, activation='elu'),
  tf.keras.layers.Dropout(0.15),
  tf.keras.layers.Dense(256*1, activation='elu'),
  tf.keras.layers.Dense(256*1, activation='elu'),
  tf.keras.layers.Dropout(0.15),
  tf.keras.layers.Dense(128, activation='elu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(64, activation='elu'),
  tf.keras.layers.Dense(32, activation='elu'),
  tf.keras.layers.Dense(19, activation='softmax') # 출력층 / 확률값 출력을 위해 softmax / sigmoid는 이진분류 / input이 20개이므로 출력층은 19
])

[Sequential Model](https://www.tensorflow.org/guide/keras/sequential_model?hl=ko)  
[Keras Dense](https://han-py.tistory.com/207)  
[activation function_1](https://m.blog.naver.com/wideeyed/221017173808)  
[activation function_2](https://seongkyun.github.io/study/2019/05/01/activations/)

In [ ]:
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',  # 손실함수, 최소화 시켜야하는 것 / 다중 클래스 분류
              metrics=['accuracy'])

In [ ]:
model.fit(train_x,
          train_y,
          batch_size=256*3, # 학습시 데이터 개수
          validation_split=0.1,  # 훈련데이터 10%를 검증데이터로 사용
          epochs=200)  # 훈련횟수가 200

Epoch 1/200
235/235 [==============================] - 3s 9ms/step - loss: 1.4669 - accuracy: 0.5753 - val_loss: 0.6865 - val_accuracy: 0.7684
Epoch 2/200
235/235 [==============================] - 2s 7ms/step - loss: 0.7080 - accuracy: 0.7627 - val_loss: 0.5750 - val_accuracy: 0.8025
Epoch 3/200
235/235 [==============================] - 2s 7ms/step - loss: 0.6349 - accuracy: 0.7817 - val_loss: 0.5321 - val_accuracy: 0.8220
Epoch 4/200
235/235 [==============================] - 2s 7ms/step - loss: 0.5917 - accuracy: 0.7984 - val_loss: 0.5398 - val_accuracy: 0.8104
Epoch 5/200
235/235 [==============================] - 2s 7ms/step - loss: 0.5616 - accuracy: 0.8089 - val_loss: 0.5146 - val_accuracy: 0.8235
Epoch 6/200
235/235 [==============================] - 2s 7ms/step - loss: 0.5340 - accuracy: 0.8173 - val_loss: 0.5109 - val_accuracy: 0.8263
Epoch 7/200
235/235 [==============================] - 2s 7ms/step - loss: 0.5422 - accuracy: 0.8157 - val_loss: 0.4715 - val_accuracy: 0.8377

In [ ]:
test = pd.read_csv('test.csv').reset_index(drop=True)
test_ids = test['id']
test = test.drop(columns=['id', 'fiberID'])
test = pd.DataFrame(scaler.transform(test), columns=test.columns, index=test.index)

In [ ]:
pred_mat = model.predict(test)

In [ ]:
sample = pd.read_csv('sample_submission.csv')

submission = pd.DataFrame(pred_mat, index=test.index)
submission = submission.rename(columns=i2lb)
submission = pd.concat([test_ids, submission], axis=1)
submission = submission[sample.columns]
submission.to_csv("submission.csv", index=False)